Spectral Clustering

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [20]:
graph=pickle.load(open('./graphs/graph_match2499719_team1609.pkl','rb'))
print(len(graph.nodes))
print(graph.nodes[40])
print(len(graph.edges))
print(graph.edges[25, 26])

39
{'event_count': 28, 'most_common_event': 'PASS', 'unique_players': 5, 'zone_name': 'OUTSIDE', 'role_distribution': array([0.2, 0.2, 0.6, 0. ])}
293
{'weight': 9, 'transition_frequency': 0.4090909090909091, 'most_common_event': 'PASS', 'start_zone_name': 'LEFT_WING_MID_THIRD_ATT', 'end_zone_name': 'LEFT_HALF_MID_THIRD_ATT'}


We have to encode all event types

In [37]:
def encode_event_semantic(event_type):
    """Group events by semantic meaning"""
    # Group related events
    attacking_events = ['PASS', 'SHOT', 'CARRY', 'TAKE_ON']
    defensive_events = ['CLEARANCE', 'INTERCEPTION', 'PRESSURE', 'RECOVERY', 'DUEL']
    goalkeeper_events = ['GOALKEEPER']
    disruption_events = ['FOUL_COMMITTED', 'CARD', 'MISCONTROL', 'BALL_OUT']
    meta_events = ['FORMATION_CHANGE', 'SUBSTITUTION', 'PLAYER_ON', 'PLAYER_OFF']
    
    if event_type in attacking_events:
        return [1, 0, 0, 0, 0, 0]  # Attacking
    elif event_type in defensive_events:
        return [0, 1, 0, 0, 0, 0]  # Defensive
    elif event_type in goalkeeper_events:
        return [0, 0, 1, 0, 0, 0]  # Goalkeeper
    elif event_type in disruption_events:
        return [0, 0, 0, 1, 0, 0]  # Disruption
    elif event_type in meta_events:
        return [0, 0, 0, 0, 1, 0]  # Meta
    else:
        return [0, 0, 0, 0, 0, 1]  # Generic/Other

In [50]:
from spectral_build_vizualization import discover_tactical_patterns

labels, nodes = discover_tactical_patterns(graph, k=4)

c:\Portable\ProgramsPortable\Anaconda\envs\graphml\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


## Silhouette score

The silhouette value is a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation). The silhouette value ranges from −1 to +1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters. If most objects have a high value, then the clustering configuration is appropriate. If many points have a low or negative value, then the clustering configuration may have too many or too few clusters. 

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
node_features = []
for node in graph.nodes():
    
    node_data = graph.nodes[node]
    
    # Create feature vector from the node attributes
    features = [
        node_data['event_count'],
        node_data['unique_players'],
        *encode_event_semantic(node_data['most_common_event']),
        *node_data['role_distribution']  # This unpacks the 4 values
    ]
    node_features.append(features)

node_features = np.array(node_features)
# Standardize features so as to have mean=0 and variance=1 
# Otherwise, features with larger scales can dominate the distance calculations
node_features = scalar.fit_transform(node_features)
score = silhouette_score(node_features, np.array(labels))
print(score)

-0.08182848837817866


## Calinski–Harabasz (CH) Index
Defined as the ratio of the between-cluster separation (BCSS) to the within-cluster dispersion (WCSS), normalized by their number of degrees of freedom. BCSS (Between-Cluster Sum of Squares) is the weighted sum of squared Euclidean distances between each cluster centroid (mean) and the overall data centroid (mean). WCSS (Within-Cluster Sum of Squares) is the sum of squared Euclidean distances between the data points and their respective cluster centroids.

- < 10: Very poor separation
- 10-50: Poor to moderate separation
- 50-100: Moderate separation
- 100+: Good separation
- 1000+: Excellent separation

In [59]:
from sklearn.metrics import calinski_harabasz_score
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
labels, nodes = discover_tactical_patterns(graph, k=4)
node_features = []
for node in graph.nodes():
    
    node_data = graph.nodes[node]
    
    # Create feature vector from the node attributes
    features = [
        node_data['event_count'],
        node_data['unique_players'],
        *encode_event_semantic(node_data['most_common_event']),
        *node_data['role_distribution']  # This unpacks the 4 values
    ]
    node_features.append(features)

node_features = np.array(node_features)
node_features = scalar.fit_transform(node_features)
ch_score = calinski_harabasz_score(node_features, np.array(labels))
print(ch_score)

3.519115642133684


c:\Portable\ProgramsPortable\Anaconda\envs\graphml\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


## Davies Bouldin Score

In [60]:
from sklearn.metrics import davies_bouldin_score
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
labels, nodes = discover_tactical_patterns(graph, k=4)
node_features = []
for node in graph.nodes():
    
    node_data = graph.nodes[node]
    
    # Create feature vector from the node attributes
    features = [
        node_data['event_count'],
        node_data['unique_players'],
        *encode_event_semantic(node_data['most_common_event']),
        *node_data['role_distribution']  # This unpacks the 4 values
    ]
    node_features.append(features)

node_features = np.array(node_features)
node_features = scalar.fit_transform(node_features)
db_score = davies_bouldin_score(node_features, np.array(labels))
print(db_score)

3.1881962153032823


c:\Portable\ProgramsPortable\Anaconda\envs\graphml\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


## ANOVA (Analysis of Variance) 
Tests whether cluster means differ significantly across features. It tells you if your clusters are actually separating the data in meaningful ways. What ANOVA Tells You:

- High F-statistic, Low p-value (<0.05): Feature significantly differs between clusters
- Low F-statistic, High p-value (>0.05): Feature doesn't help distinguish clusters

In [62]:
from scipy import stats
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
labels, nodes = discover_tactical_patterns(graph, k=4)
node_features = []
for node in graph.nodes():
    
    node_data = graph.nodes[node]
    
    # Create feature vector from the node attributes
    features = [
        node_data['event_count'],
        node_data['unique_players'],
        *encode_event_semantic(node_data['most_common_event']),
        *node_data['role_distribution']  # This unpacks the 4 values
    ]
    node_features.append(features)

node_features = np.array(node_features)
node_features = scalar.fit_transform(node_features)

def anova_test_clustering(features, labels, feature_names=None):
    """
    Perform ANOVA test for each feature across clusters
    
    Args:
        features: numpy array (n_samples, n_features)
        labels: cluster labels
        feature_names: list of feature names
    
    Returns:
        DataFrame with F-statistics and p-values
    """
    n_features = features.shape[1]
    
    if feature_names is None:
        feature_names = [f'Feature_{i}' for i in range(n_features)]
    
    results = []
    
    for i in range(n_features):
        # Split feature values by cluster
        groups = [features[labels == cluster, i] for cluster in np.unique(labels)]
        
        # Perform one-way ANOVA
        f_stat, p_value = stats.f_oneway(*groups)
        
        results.append({
            'Feature': feature_names[i],
            'F-statistic': f_stat,
            'p-value': p_value,
            'Significant': 'Yes' if p_value < 0.05 else 'No'
        })
    
    return pd.DataFrame(results).sort_values('F-statistic', ascending=False)

feature_names = (
    ['event_count', 'unique_players'] + 
    [f'semantic_{i}' for i in range(6)] + 
    [f'role_{i}' for i in range(4)]
)

anova_results = anova_test_clustering(node_features, labels, feature_names)
print(anova_results.to_string(index=False))

# Summary
significant_features = anova_results[anova_results['Significant'] == 'Yes']
print(f"\n{len(significant_features)}/{len(feature_names)} features significantly differ between clusters")

       Feature  F-statistic      p-value Significant
unique_players    17.684085 3.705726e-07         Yes
   event_count    14.670960 2.380039e-06         Yes
    semantic_0     2.852519 5.119929e-02          No
    semantic_1     2.852519 5.119929e-02          No
        role_2     2.656236 6.351865e-02          No
        role_1     0.877082 4.623076e-01          No
        role_0     0.512427 6.763652e-01          No
        role_3     0.046575 9.864417e-01          No
    semantic_2          NaN          NaN          No
    semantic_3          NaN          NaN          No
    semantic_4          NaN          NaN          No
    semantic_5          NaN          NaN          No

2/12 features significantly differ between clusters


c:\Portable\ProgramsPortable\Anaconda\envs\graphml\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Portable\ProgramsPortable\Anaconda\envs\graphml\Lib\site-packages\scipy\stats\_axis_nan_policy.py:579: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)


## Modularity

Modularity measures how well a network is divided into communities/clusters. It compares the number of edges within clusters vs what you'd expect in a random network.

Range: -1 to 1

- \> 0.3: Good community structure
- \> 0.5: Strong community structure
- \> 0.7: Very strong community structure
- < 0.3: Weak or no community structure
Negative: Worse than random

Key difference: Unlike Silhouette/CH, modularity uses the graph structure (edges), not just node features!

In [63]:
import networkx as nx

labels, nodes = discover_tactical_patterns(graph, k=4)

def calculate_modularity(graph, labels):
    """
    Calculate modularity for a clustering on a graph
    
    Args:
        graph: NetworkX graph (DiGraph or Graph)
        labels: cluster assignment for each node
    
    Returns:
        modularity score
    """
    # Create communities from labels
    # NetworkX expects a list of sets, one set per community
    unique_labels = np.unique(labels)
    node_list = list(graph.nodes())
    
    communities = []
    for label in unique_labels:
        # Get nodes in this cluster
        cluster_nodes = [node_list[i] for i, l in enumerate(labels) if l == label]
        communities.append(set(cluster_nodes))
    
    # Calculate modularity
    modularity = nx.community.modularity(graph, communities)
    
    return modularity

modularity_score = calculate_modularity(graph, labels)
print(f"Modularity Score: {modularity_score}")

Modularity Score: 0.32949708034846625


c:\Portable\ProgramsPortable\Anaconda\envs\graphml\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


# More graph-based metrics
### Conductance : Measures cluster boundary quality (lower = tighter clusters)

Formula: boundary_edges / (internal_edges + boundary_edges)
- Interpretation: < 0.3 is good, < 0.1 is excellent

### Coverage: Fraction of edges within clusters (higher = better)

Formula: internal_edges / total_edges
- Interpretation: > 0.7 is good, > 0.9 is excellent


### Internal Edge Density: Average density within clusters

Shows how tightly connected nodes are within their clusters
Higher values indicate more cohesive clusters

In [64]:
def compute_conductance(graph, labels):
    """Compute conductance for each cluster (lower is better)"""
    node_list = list(graph.nodes())
    conductances = []
    
    for label in np.unique(labels):
        cluster_nodes = set([node_list[i] for i, l in enumerate(labels) if l == label])
        
        # Count edges within and across cluster boundary
        internal_edges = 0
        boundary_edges = 0
        
        for u, v in graph.edges():
            if u in cluster_nodes and v in cluster_nodes:
                internal_edges += 1
            elif u in cluster_nodes or v in cluster_nodes:
                boundary_edges += 1
        
        # Conductance = boundary / min(internal, external)
        if internal_edges + boundary_edges > 0:
            conductance = boundary_edges / (internal_edges + boundary_edges)
        else:
            conductance = 1.0
        
        conductances.append(conductance)
    
    return np.mean(conductances)



In [65]:
def compute_coverage(graph, labels):
    """Compute coverage: fraction of edges within clusters"""
    node_list = list(graph.nodes())
    total_edges = graph.number_of_edges()
    internal_edges = 0
    
    for u, v in graph.edges():
        u_idx = node_list.index(u)
        v_idx = node_list.index(v)
        if labels[u_idx] == labels[v_idx]:
            internal_edges += 1
    
    return internal_edges / total_edges if total_edges > 0 else 0

In [66]:
def compute_edge_density_ratio(graph, labels):
    """Ratio of internal to external edge density"""
    node_list = list(graph.nodes())
    
    internal_density = []
    for label in np.unique(labels):
        cluster_nodes = [node_list[i] for i, l in enumerate(labels) if l == label]
        if len(cluster_nodes) > 1:
            subgraph = graph.subgraph(cluster_nodes)
            internal_density.append(nx.density(subgraph))
    
    return np.mean(internal_density) if internal_density else 0

In [67]:
conductance = compute_conductance(graph, labels)
coverage = compute_coverage(graph, labels)
edge_density = compute_edge_density_ratio(graph, labels)

print(f"Conductance: {conductance}")
print(f"Coverage: {coverage}")
print(f"Edge Density Ratio: {edge_density}")

Conductance: 0.6664031386938685
Coverage: 0.5119453924914675
Edge Density Ratio: 0.5967643467643468


## Pairwise Comparisons

Using normalised mutual information and adjusted random index

### Normalized Mutual Information (NMI): Measures agreement between two clusterings

Range: 0-1 (1 = perfect agreement)

Use: Compare how similar different methods' results are

### Adjusted Rand Index (ARI):

Range: -1 to 1 (1 = perfect agreement, 0 = random)

Adjusts for chance agreement

In [68]:
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score